In [1]:
import onnx_graphsurgeon as gs
import onnx
import numpy as np
import sys
sys.path.append("..")
import onnx_utils
import add_rewrite

In [2]:
graph = gs.import_onnx(onnx.load("/home/yssun/onnx/enode/test_model/model_identity.onnx"))

In [3]:
graph.nodes[0]

onnx_graphsurgeon_node_2 (Add)
	Inputs: [
		Variable (input): (shape=[1024, 22], dtype=int64)
		Constant (onnx_graphsurgeon_constant_0): (shape=[1024, 22], dtype=int64)
	]
	Outputs: [
		Variable (add_out_gs_1): (shape=[1024, 22], dtype=int64)
	]

In [4]:
addNode = graph.nodes[0]

In [5]:
addNode.inputs

[Variable (input): (shape=[1024, 22], dtype=int64),
 Constant (onnx_graphsurgeon_constant_0): (shape=[1024, 22], dtype=int64)
 LazyValues (shape=[1024, 22], dtype=int64)]

In [6]:
addVarInput = addNode.inputs[0]
addConstInput = addNode.inputs[1]

In [ ]:
reOp = graph.addRedencyRewrite(graph1.nodes[0].inputs[0],'/test',
                                12,22,1,np.ones((1024,10),dtype=np.int64),0,12,0,np.array([1024,1]),np.ones((1,12),dtype=np.int64),[0])

In [7]:
reop = graph.addRedencyRewrite(addVarInput,addNode,addConstInput,
                               12,22,0,12)

In [8]:
graph.outputs.append(reop)

In [9]:
for out in graph.outputs:
    out.dtype = np.int64

In [10]:
model = gs.export_onnx(graph)


In [11]:
onnx.save(model, "../test_model/re_model_identity.onnx")

In [12]:
from onnxruntime import SessionOptions,ExecutionMode
import onnxruntime as ort
sess_opt = SessionOptions()
sess_opt.execution_mode  = ExecutionMode.ORT_PARALLEL
# sess_opt.inter_op_num_threads = 3
session = ort.InferenceSession("../test_model/re_model_identity.onnx",sess_options = sess_opt,providers=[ 'CPUExecutionProvider'])
for i in range(10):
  random_input1 = np.random.rand(1024,22).astype(np.int64)
  out = session.run([],{"input":random_input1})

In [13]:
out

[array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=int64),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=int64)]

In [21]:
x = np.random.randint(0, 100, size=(1024, 22), dtype=np.int64)


In [22]:
x[:, 0:12] = x[0, 0:12]


In [23]:
x

array([[86, 68, 41, ..., 61, 89,  0],
       [86, 68, 41, ..., 21, 16,  9],
       [86, 68, 41, ..., 41, 53, 56],
       ...,
       [86, 68, 41, ..., 11, 72, 36],
       [86, 68, 41, ..., 41, 57, 91],
       [86, 68, 41, ..., 56,  2, 45]])

In [24]:
out = session.run([],{"input":x})

In [25]:
out

[array([[87, 69, 42, ..., 62, 90,  1],
        [87, 69, 42, ..., 22, 17, 10],
        [87, 69, 42, ..., 42, 54, 57],
        ...,
        [87, 69, 42, ..., 12, 73, 37],
        [87, 69, 42, ..., 42, 58, 92],
        [87, 69, 42, ..., 57,  3, 46]], dtype=int64),
 array([[87, 69, 42, ..., 62, 90,  1],
        [87, 69, 42, ..., 22, 17, 10],
        [87, 69, 42, ..., 42, 54, 57],
        ...,
        [87, 69, 42, ..., 12, 73, 37],
        [87, 69, 42, ..., 42, 58, 92],
        [87, 69, 42, ..., 57,  3, 46]], dtype=int64)]

In [28]:
x1 = out[0]

In [29]:
x2 = out[1]

In [30]:
x1_slice = x1[:,:12]
x2_slice = x2[:,:12]

In [31]:
x1_slice

array([[87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21],
       ...,
       [87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21]], dtype=int64)

In [32]:
x2_slice

array([[87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21],
       ...,
       [87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21],
       [87, 69, 42, ..., 67, 24, 21]], dtype=int64)

In [33]:
x2_slice == x1_slice

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])